# WEB SCRAPPING
## Data mining from Peruvian Open Source Data content provided by the Goverment

### Scope of this Notebook
[Datos Abiertos]("https://www.datosabiertos.gob.pe") is a raw open data source provide by the goverment of Peru.
I used webscrapping methods in **Python** to obtain relevant data from a *energy* sector.
At the end of the nombre, a dataset is provided containing the results of the webscrapping and data mining.
This is just a taste of what I am able to do for collecting data from web pages.

In [1]:
# this module helps in web scrapping
from bs4 import BeautifulSoup
# this module helps us to download a web page
import requests

import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def replace_labels(df):
    
    for i, key in enumerate(df.keys()):
        if ("Dataset" in key) or ("File" in key):
            continue
        else:
            df.rename(columns={key: "Dataset " + key}, inplace=True)
            
    df.rename(columns={"Dataset Fecha de lanzamiento": "Dataset Release Date",
                       "Dataset Fecha modificada ": "Dataset Modified Date"},
              inplace=True)
      
    return df

In [3]:
print("Insert the http address of the sector you want to analyse:")
http_address = input()

Insert the http address of the sector you want to analyse:


 https://www.datosabiertos.gob.pe/SEARCH/field_topic/energ%C3%ADa-340?sort_by=changed


In [4]:
main_address = "https://www.datosabiertos.gob.pe/"
search_field = "SEARCH/field_topic/"
subfix_field = "?sort_by=changed"

sector_field = http_address.replace(main_address, "").replace(search_field, "").replace(subfix_field, "")

In [5]:
data  = requests.get(http_address).text
soup = BeautifulSoup(data, "html.parser")

n_dataset_in_sector = int(re.search(r'\d+', soup.find(class_="view-header").text).group())

In [6]:
#data files in sector
n_links_per_page = 10
n_pages = int(np.ceil(n_dataset_in_sector/n_links_per_page))

print(f"The category {sector_field.upper()} has {n_dataset_in_sector} data files")
print(f"Only {n_links_per_page} links are shown per page")
print(f"There is a total of {n_pages} pages in {sector_field.upper()}")

The category ENERG%C3%ADA-340 has 36 data files
Only 10 links are shown per page
There is a total of 4 pages in ENERG%C3%ADA-340


In [7]:
# For page 1 with 10 results (35 results for Energy, 4 browsing pages)
# getting all URLs
URL = []

for i in range(n_pages):
    query = f"?query=&sort_by=changed&sort_order=DESC&page=0%2C{i}"
    URL.append(main_address + search_field + sector_field + query)

In [8]:
children = []

for url in tqdm(URL, desc="Pages scanned"):
    data  = requests.get(url).text
    soup = BeautifulSoup(data, "html.parser")
    tag_view_content = soup.body.find_all(class_="view-content")[0]

    for child in tag_view_content.children:
        if child != '\n':
            children.append(child)

Pages scanned: 100%|██████████████████████████████| 4/4 [00:03<00:00,  1.01it/s]


In [9]:
print(f"The total number of files to be download are: {len(children)}")

The total number of files to be download are: 36


In [10]:
df_meta_data = pd.DataFrame(columns=["Dataset Order",
                                     "Dataset Title",
                                     "Dataset Link",
                                     "Dataset Produced By",
                                     "Dataset Description",])

for i, child in tqdm(enumerate(children), total=len(children), desc="Processing Metadata per link"):    
    item = {}
    
    item["Dataset Order"] = f"N_{i:02d}"
    
    try:
        item["Dataset Title"] = child.find_all(class_="node-title")[0].text
    except IndexError or AttributeError:
        item["Dataset Title"] = None
    
    try:
        item["Dataset Link"] = main_address + child.h2.a["href"]
    except IndexError or AttributeError:
        item["Dataset Link"] = None

    try:
        item["Dataset Produced By"] = child.find_all(class_="group-membership")[0].text
    except IndexError or AttributeError:
        item["Dataset Produced By"] = None
    
    try:
        item["Dataset Description"] = child.find_all(class_="node-description")[0].p.text
    except AttributeError:
        item["Dataset Description"] = None
    
    df_meta_data = pd.concat([df_meta_data, pd.DataFrame([item])], ignore_index=True)
    
del soup

Processing Metadata per link: 100%|████████████| 36/36 [00:00<00:00, 820.82it/s]


In [11]:
# Getting the missing labels for the meta data of each file
# They are extracted from the specific page of each dataset inside the energy sector
labels = []
for link in tqdm(df_meta_data["Dataset Link"], desc="Scanning for unique labels"):
    data = requests.get(link).text
    soup = BeautifulSoup(data, "html.parser")  # create a soup object using the variable 'data'
    
    for label in soup.table.find_all(class_="field-label"):
        labels.append(label.text)
        
labels = list(set(labels))

Scanning for unique labels: 100%|███████████████| 36/36 [00:39<00:00,  1.09s/it]


In [12]:
df_meta_data = pd.concat([df_meta_data, pd.DataFrame(None,
                                                     index=range(n_dataset_in_sector),
                                                     columns=list(labels))], axis=1)

In [13]:
for n, row in tqdm(df_meta_data.iterrows(), total=len(df_meta_data), desc="Processing Rows"):
    data = requests.get(row["Dataset Link"]).text
    # Create a soup object using the variable 'data'
    soup = BeautifulSoup(data, "html.parser")
    table = soup.find(class_="field-group-format group_additional")
    for i, tr in enumerate(table.find_all("tr")):
        if i == 0:
            continue
  
        for label in labels:
            if label in tr.text:
                df_meta_data.at[n, label] = tr.text.split(label)[1]
            else:
                continue

Processing Rows: 100%|██████████████████████████| 36/36 [00:51<00:00,  1.43s/it]


In [14]:
df_files_data = pd.DataFrame(columns=["Dataset Link", "File Title", "File Description", "File Format", "File Link"])

In [15]:
for url in tqdm(df_meta_data["Dataset Link"], desc="Getting files info per Dataset"):
    data = requests.get(url).text
    soup = BeautifulSoup(data, "html.parser")
    tag_view_content = soup.body.find(class_="resource-list")
    
    item = {}    
    
    try:
        # Since the last row is always empty in "Dato y Medio de Distribución" file list, unless there is only one file.
        if len(tag_view_content.find_all("li")) == 1:
            files_container = tag_view_content.find_all("li")
        else:
            files_container = tag_view_content.find_all("li")[:-1]
        
        for tag in files_container:
            
            item["Dataset Link"] = url
        
            try:
                item["File Title"] = tag.a["title"]
            except AttributeError:
                item["File Title"] = None

            try:
                item["File Description"] = tag.p.p.text
            except AttributeError:
                item["File Description"] = None

            try:
                item["File Format"] = tag.a.span["data-format"]
            except AttributeError:
                item["File Format"] = None
                
            try:
                item["File Link"] = tag.find(class_="btn btn-primary data-link")["href"]
            except AttributeError:
                item["File Link"] = None
        
            df_files_data = pd.concat([df_files_data, pd.DataFrame([item])], ignore_index=True)
            
    except:
        item = {url, None, None, None, None}
        df_files_data = pd.concat([df_files_data, pd.DataFrame([item])], ignore_index=True)

Getting files info per Dataset: 100%|███████████| 36/36 [00:42<00:00,  1.17s/it]


In [16]:
df_result = pd.merge(df_meta_data, df_files_data, how="inner", on=["Dataset Link", "Dataset Link"])

In [18]:
df_meta_data = replace_labels(df_meta_data)

In [19]:
df_result = replace_labels(df_result)

In [20]:
label_order = ['Dataset Order',
               'Dataset Title',
               'Dataset Link',
               'Dataset Produced By',
               'Dataset Description',
               'Dataset Identificador',
               'Dataset Publisher',
               'Dataset Author',
               'Dataset Contact Name',
               'Dataset Contact Email',
               'Dataset Release Date',
               'Dataset Modified Date',
               'Dataset Temporal Coverage',
               'Dataset Frequency',
               'Dataset Public Access Level',
               'Dataset Homepage URL',
               'Dataset Language',
               'Dataset License',
               'File Title',
               'File Description',
               'File Format',
               'File Link']

In [21]:
df_result = df_result[label_order]

In [22]:
df_meta_data.to_csv("df_meta_data.csv")
df_files_data.to_csv("df_files_data.csv")
df_result.to_csv("df_result.csv")